In [122]:
import bs4
import base64
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import pymystem3 as ms
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from urllib.parse import urljoin 

In [56]:
def tokenize(s):
    #print(s)
    s = ' '.join(re.split('https?://.*\\W', s))
    return list(filter(lambda x: len(x) > 1, re.split('\\W', s)))

In [8]:
def text_from_doc(doc):
    a = []
    b = []
    #print(type(doc))

    if isinstance(doc, bs4.element.Tag):
        for child in doc.contents:
            kek = text_from_doc(child)
            a += kek[0]
            b += kek[1]
        if doc.name == 'a':
            if doc.has_attr('href'):
                b.append(doc['href'])
    else:
        a += tokenize(doc)
    return a, b

In [104]:
mystem = ms.Mystem()
wnl = WordNetLemmatizer()

In [115]:
doc_f = open('docs.out', 'w')
lem_f = open('lemmas.out', 'w')

freqs = dict()
idfs = dict()

for doc_id in range(10):
    file_name = f'../byweb_for_course/byweb.{doc_id}.xml'
    with open(file_name, 'r') as inf:
        inf.readline()
        for i in tqdm(range(20000)):
            a = inf.readline()
            b = inf.readline()
            c = inf.readline()

            docid = re.split('<.*?>', c)[1]
            url = base64.b64decode(re.split('<.*?>', b)[1]).decode("cp1251")
            content = base64.b64decode(re.split('<.*?>', a)[2]).decode("cp1251")
            soup1 = bs4.BeautifulSoup(content, 'lxml')

            for tag in soup1(['script', 'style']):
                tag.decompose()
            
            a = soup1.get_text()
            a = re.sub('(<!--.*?-->)', '', a, flags=re.DOTALL)
            a = re.sub('<.*?>', '', a, flags=re.DOTALL)
            a = ' '.join(tokenize(a))
            lm = ''.join(mystem.lemmatize(a))
            lm = ' '.join([(wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i)) for i,j in pos_tag(list(map(lambda x: x.lower(), word_tokenize(lm))))])
            
            
            #a = []
            #b = []
            #for doc in soup1:
            #    kek = text_from_doc(doc)
            #    a += kek[0]
            #    b += kek[1]

            
            
            #jn = ' '.join(a)
            #lemmas = !cat a | /home/mikhail/mystem
            #lemmas = lemmas[0]
            #tokens = re.split('{|}', lemmas)
            #tokens1 = []
            #for i in range(1, len(tokens), 2):
            #    tokens1.append(re.split('\|', tokens[i])[0])

            #idf = dict()

            #for t in tokens1:
            #    idf[t] = 1
            #    if t not in freqs:
            #        freqs[t] = 1
            #    else:
            #        freqs[t] = freqs[t] + 1

            #for k in idf.keys():
            #    idfs[k] += idf[k]

            
            doc_f.write('{}\t{}\t{}\n'.format(docid, url, a))
            lem_f.write('{}\t{}\t{}\n'.format(docid, url, lm))
    
            
            
            #wordlen_f.write('{}\n'.format(len(a)))
            #bytelen_f.write('{}\n'.format(len(soup1.get_text())))
        
doc_f.close()
lem_f.close()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

In [121]:
np.average(bytelens)

199280.0

In [122]:
np.average(wordlens)

70.92592592592592

In [9]:
kek = dict()
inf = open('wordlens.out', 'r')
s = inf.readline()
while len(s) > 0:
    ln = int(s)
    key = (ln // 100) * 100
    if key not in kek:
        kek[key] = 1
    else:
        kek[key] = kek[key] + 1
    s = inf.readline()

In [10]:
df = pd.DataFrame()
df["n_docs"] = kek.keys()
df["len in words"] = kek.values()

px.bar(df, x="n_docs", y="len in words")

In [31]:
inf = open('bytelens.out', 'r')
inf1 = open('wordlens.out', 'r')
s = inf.readline()
sm = 0
ct = 0
while len(s) > 0:
    s1 = inf1.readline()
    ln = int(s)
    ln1 = int(s1)
    if (ln == 0 and ln1 != 0):
        print(ct)
    sm += 1 if (ln == 0) else ln1 / ln
    ct += 1
    s = inf.readline()
print('n_docs: {}, average len in words: {}'.format(ct, sm / ct))

n_docs: 200000, average len in words: 0.01204361733264265


In [8]:
kek = dict()
inf = open('bytelens.out', 'r')
s = inf.readline()
while len(s) > 0:
    ln = int(s)
    key = (ln // 10000) * 10000
    if key not in kek:
        kek[key] = 1
    else:
        kek[key] = kek[key] + 1
    s = inf.readline()
    
df = pd.DataFrame()
df["n_docs"] = kek.keys()
df["len in bytes"] = kek.values()

px.bar(df, x="n_docs", y="len in bytes")

In [106]:
a = ' '.join(docs[7][2])
b = !echo $a | /home/mikhail/mystem

In [91]:
docs[7][1]

'http://brest.linux.by/forum/index.php?action=vthread&forum=8&topic=126'

In [56]:
s = 'abacaba http://krk.lol/arbidol caa'
re.split('https?://.*\\W', s)

['abacaba ', 'caa']

In [92]:
re.split('({|})', b[0])

['Начало']

In [93]:
b[0]

'Начало'

In [94]:
re.split('(^|}).*?{', b[0])

['Начало']

In [107]:
b

['Начало{начало|начинать}Ответить{отвечать}Статистика{статистика|статистик}Регистрация{регистрация}Поиск{поиск}FAQ{FAQ??}Язык{язык}Аноним{аноним}thxburjuinrwz{thxburjuinrwz??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}']

In [117]:
kek = re.split('{|}', b[0])
kek1 = ''
for i in range(1, len(kek), 2):
    kek1 += re.split('\|', kek[i])[0] + ' '

In [118]:
kek1

'начало отвечать статистика регистрация поиск FAQ?? язык аноним thxburjuinrwz?? html?? html?? html?? html?? html?? html?? html?? аноним mziwj?? html?? html?? html?? html?? html?? html?? html?? html?? аноним mziwj?? '

''

In [27]:
idfs

{}

In [3]:
inf = open('docs.out', 'r')
ouf = open('lemmas.out', 'w')
s = inf.readline()
ct = 0
while len(s) > 0:
#for i in range(10):
    ct += 1
    if ct > 500:
        print('500')
        ct = 0
    word = s.split('\t')[2]
    words = word[1:len(word) - 1].split("', '")
    words[0] = words[0][1:]
    words[-1] = words[-1][:-1]
    ouf1 = open('tmp', 'w')
    concat = ' '.join(words)
    ouf1.write(concat)
    ouf1.close()
    lemmas = !cat tmp | /home/mikhail/mystem -lnd
    ouf.write('{}\n'.format(lemmas))
    #print(lemmas)
    
    s = inf.readline()
    
ouf.close()
    

500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500


In [35]:
ms.lemmatize('lol кек чебуреков')

['lol', ' ', 'кек', ' ', 'чебурек', '\n']

In [116]:
!head lemmas.out -n 20

4	http://forum.linux.by/viewtopic.php?t=6497	linux by просмотр тема хотеть что бы аська сам переконекчиваться
11	http://forum.linux.by/viewtopic.php?t=928	linux by просмотр тема помогать
29	http://forum.linux.by/viewtopic.php?t=3739	linux by просмотр тема zyxel omni 56k plus usb как заставлять он работать
31	http://forum.linux.by/viewtopic.php?t=2416	linux by просмотр тема обзор mandrake linux 10 community edition
34	http://forum.linux.by/viewtopic.php?p=32833	linux by просмотр тема открытый занятие по unix часть продолжение
47	http://forum.linux.by/viewtopic.php?p=55398&sid=6a4b28a076ba11dad80a5f53475a35fa	linux by просмотр тема быть ли кто опыт установка ncpi под wine
54	http://brest.linux.by/forum/index.php?action=vthread&forum=8&topic=136&page=2	форум blug sfgwg начинать отвечать статистика регистрация поиск faq язык форум blug test sfgwg посл ответ сообщение аноним дата 25 мара 2007 20 27 57buy levitra buy levitra аноним дата 26 мара 2007 10 30 53 phentermine without prescription 

In [55]:
a = "мама мыла раму"
lemmas = !echo $a | /home/mikhail/mystem -lnd

In [56]:
lemmas

['мама', 'мыть', 'рама']

In [28]:
!head wordlens.out -n 3683

In [24]:
re.sub('a', 'b', 'cac', flags=re.DOTALL)

'cbc'

In [37]:
''.join(ms.lemmatize(' '.join(tokenize(soup1.get_text()))))

'оплата карта Visa MasterCard мтс абонент главный карта сайт поиск bel оплата карта Visa MasterCard центр обслуживание абонент мтс УП иностранный банк москва минск АСБ беларусбанк оао белвнешэкономбанк оао белпромстройбанк оао приорбанк оао белинвестбанк оао технобанк зао межторгбанк зао трастбанк зао белсвиссбанк зао минский транзитный банк оао паритетбанк оплата карта система белкарт банкомат оплата карта система нефтекарт банкомат абонент как платить оплата банковский карта оплата карта Visa MasterCard оплата карта Visa MasterCard центр обслуживание абонент мтс УП иностранный банк москва минск АСБ беларусбанк оао белвнешэкономбанк оао белпромстройбанк оао белинвестбанк приорбанк оао оао технобанк зао белсвиссбанк зао трастбанк зао межторгбанк зао минский транзитный банк оао паритетбанк узнавать тариф на GPRS роуминг узнавать как совершать международный звонок добавлять услуга по SMS высказывать свой мнение работа сеть мтс узнавать новый тариф на международный роуминг входить интерне

In [71]:
wnl = WordNetLemmatizer()

In [111]:
txt = 'Pigeons are жопой running aT tHe valleys\n'
[wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(list(map(lambda x: x.lower(), word_tokenize(txt))))]

['pigeon', 'be', 'жопой', 'run', 'at', 'the', 'valley']

In [120]:
!pip3 install urlparse

  ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


['pigeons', 'are', 'running', 'at', 'the', 'valleys']

In [97]:
mystem.lemmatize('пощуком вібраваць чириком')

['пощук', ' ', 'вібраваць', ' ', 'чирик', '\n']

In [90]:
re.split('\W', 'перапаўненню')

['перапаўненню']

In [99]:
'KEK'.lower()

'kek'

In [127]:
urls_f = open('urls.doc', 'w')

freqs = dict()
idfs = dict()

for doc_id in range(10):
    file_name = f'../byweb_for_course/byweb.{doc_id}.xml'
    with open(file_name, 'r') as inf:
        inf.readline()
        for i in tqdm(range(20000)):
            a = inf.readline()
            b = inf.readline()
            c = inf.readline()

            docid = re.split('<.*?>', c)[1]
            url = base64.b64decode(re.split('<.*?>', b)[1]).decode("cp1251")
            content = base64.b64decode(re.split('<.*?>', a)[2]).decode("cp1251")
            soup1 = bs4.BeautifulSoup(content, 'lxml')

            urls = []
            for tag in soup1(['a']):
                if tag.has_attr('href'):
                    try:
                        urls.append(urljoin(url, tag['href']))
                    except:
                        print('oops: ', tag['href'])
            
            urls = ' '.join(urls)
            
            #a = []
            #b = []
            #for doc in soup1:
            #    kek = text_from_doc(doc)
            #    a += kek[0]
            #    b += kek[1]

            
            
            #jn = ' '.join(a)
            #lemmas = !cat a | /home/mikhail/mystem
            #lemmas = lemmas[0]
            #tokens = re.split('{|}', lemmas)
            #tokens1 = []
            #for i in range(1, len(tokens), 2):
            #    tokens1.append(re.split('\|', tokens[i])[0])

            #idf = dict()

            #for t in tokens1:
            #    idf[t] = 1
            #    if t not in freqs:
            #        freqs[t] = 1
            #    else:
            #        freqs[t] = freqs[t] + 1

            #for k in idf.keys():
            #    idfs[k] += idf[k]

            
            
    
            urls_f.write(f'{id}\t{url}\t{urls}\n')
            
            #wordlen_f.write('{}\n'.format(len(a)))
            #bytelen_f.write('{}\n'.format(len(soup1.get_text())))
        
urls_f.close()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

...

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

ValueError: Invalid IPv6 URL

In [128]:
soup1(['a'])

[<a class="menu" href="http://www.frisbee.by/index.php"><font class="f12b">Главная</font></a>,
 <a class="menu" href="http://www.frisbee.by/manua1/"><font class="f12b">Учебник</font></a>,
 <a class="menu" href="http://www.frisbee.by/tourn.php"><font class="f12b">Турниры</font></a>,
 <a class="menu" href="http://www.frisbee.by/image.php"><font class="f12b">Фотогалерея</font></a>,
 <a class="menu" href="http://www.frisbee.by/links.php"><font class="f12b">Ссылки</font></a>,
 <a class="menu" href="http://www.frisbee.by/diary.php"><font class="f12b">Дневник</font></a>,
 <a class="menu" href="http://www.frisbee.by/f/index.php"><font class="f12b">Форум</font></a>,
 <a class="menu" href="http://www.frisbee.by/chat/index.php"><font class="f12b">Чат</font></a>,
 <a class="menu" href="http://www.frisbee.by/players/"><font class="f12b">МЫ:)</font></a>,
 <a class="menu" href="http://www.frisbee.by/brest07/"><font class="f12b" color="white">БРЕСТ 2007</font></a>,
 <a href="image.php?album=lviv06&amp

In [142]:
for tag in soup1(['a']):
    if tag.has_attr('href'):
        #print(url, tag['href'])
        try:
            urls.append(urljoin(url, tag['href']))
        except:
            print('oops: ', tag['href'])

oops:  http://parishilton.happyhost.org]paris


In [140]:
try:
    urljoin(url, 'http://parishilton.happyhost.org]paris')
except:
    print('oops: ', 'http://parishilton.happyhost.org]paris')

oops:  http://parishilton.happyhost.org]paris
